In [2]:
import pandas as pd
import tqdm
from Bio import SeqIO
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# import keras functions
# import tensorflow
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Reshape, Lambda, Embedding
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import numpy as np
from Bio import SeqIO
from numpy import array
from numpy import argmax
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.backend import expand_dims
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

2022-08-03 19:17:16.862702: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-03 19:17:16.867251: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 19:17:16.867264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def get_input_for_embedding(fasta_file):
    encodings = []
    
    # define universe of possible input values
    alphabet = 'ARNDCQEGHILKMFPSTWYV-'
    
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        data = seq_record.seq
        for char in data:
            if char not in alphabet:
                return
        integer_encoded = [char_to_int[char] for char in data]
        encodings.append(integer_encoded)
    encodings = np.array(encodings)
    return encodings

In [4]:
# convert sequences to integer encoding, for embedding
test_positive_embedding = get_input_for_embedding('data/Big_testset/fasta/test_positive_sites.fasta')
test_negative_embedding = get_input_for_embedding('data/Big_testset/fasta/test_negative_sites.fasta')
train_positive_embedding = get_input_for_embedding('data/DeepSuccinylSite/fasta/positive_sites.fasta')
train_negative_embedding = get_input_for_embedding('data/DeepSuccinylSite/fasta/negative_sites.fasta')

# create labels
train_positive_labels = np.ones(train_positive_embedding.shape[0])
train_negative_labels = np.zeros(train_negative_embedding.shape[0])
test_positive_labels = np.ones(test_positive_embedding.shape[0])
test_negative_labels = np.zeros(test_negative_embedding.shape[0])

# stack positive and negative data together
X_train_full_embedding = np.vstack((train_positive_embedding,train_negative_embedding))
X_test_embedding = np.vstack((test_positive_embedding,test_negative_embedding))
y_train_full = np.concatenate((train_positive_labels, train_negative_labels), axis = 0)
y_test = np.concatenate((test_positive_labels, test_negative_labels), axis = 0)

train_positive_pt5 = pd.read_csv("data/DeepSuccinylSite/features/full/train_positive_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
train_negative_pt5 = pd.read_csv("data/DeepSuccinylSite/features/full/train_negative_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
test_positive_pt5 = pd.read_csv("data/Big_testset/ProtT5_features/test_positive_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]
test_negative_pt5 = pd.read_csv("data/Big_testset/ProtT5_features/test_negative_big_ProtT5-XL-UniRef50.csv", header = None).iloc[:,2:]

# stack positive and negative data together
X_train_pt5_full = np.vstack((train_positive_pt5,train_negative_pt5))
X_test_pt5 = np.vstack((test_positive_pt5,test_negative_pt5))


### Load trained base models

In [55]:
# load models from file
def load_all_models(model_names):
    all_models = list()
    for model in model_names:
        filename = 'selected_models/'+ model + '.h5'
        model = load_model(filename)
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

# meta learner
def define_stacked_model(members):
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            layer.trainable = False
    ensemble_visible = [model.input for model in members]
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(16, activation='relu', name = 'ds_1')(merge)
    hidden = Dense(4, activation='relu', name = 'ds_2')(hidden)
    output = Dense(1, activation='sigmoid', name = 'ds_4')(hidden)
    model = Model(inputs = ensemble_visible, outputs = output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fit a model
def fit_stacked_model(model, inputX, inputy):
    X = [inputX for _ in range(len(model.input))]
    inputy_enc = to_categorical(inputy)
    model.fit(X, inputy_enc, epochs=10, verbose=1)   

# prediction
def predict_stacked_model(model, inputX):
    X = [inputX for _ in range(len(model.input))]
    return model.predict(X, verbose=0)


In [91]:
# load all models
ANN = load_model("selected_models/ANN.h5")
Embedding = load_model("selected_models/Embedding.h5")

# create truncated models
ANN_new = Model(inputs=ANN.input, outputs=ANN.get_layer(index=4).output)

Embedding_new = Model(inputs=Embedding.input, outputs=Embedding.get_layer(index=6).output)


stacked_model = define_stacked_model([Embedding_new, ANN_new])

# fit_stacked_model(stacked_model, x_test, y_test)
st_history = stacked_model.fit([X_train_full_embedding, X_train_pt5_full], y_train_full, epochs=7, verbose=1)

Epoch 1/7
297/297 [==============================] - 3s 7ms/step - loss: 0.3264 - accuracy: 0.8651
Epoch 2/7
297/297 [==============================] - 2s 7ms/step - loss: 0.2587 - accuracy: 0.8949
Epoch 3/7
297/297 [==============================] - 2s 7ms/step - loss: 0.2430 - accuracy: 0.9015
Epoch 4/7
297/297 [==============================] - 2s 7ms/step - loss: 0.2425 - accuracy: 0.8994
Epoch 5/7
297/297 [==============================] - 2s 7ms/step - loss: 0.2321 - accuracy: 0.9085
Epoch 6/7
297/297 [==============================] - 2s 7ms/step - loss: 0.2316 - accuracy: 0.9054
Epoch 7/7
297/297 [==============================] - 2s 7ms/step - loss: 0.2348 - accuracy: 0.9042


In [92]:
# performance evaluation
y_pred_1 = stacked_model.predict([X_test_embedding, X_test_pt5])
y_pred = (y_pred_1 > 0.5)
y_pred = [int(i) for i in y_pred]
y_test = np.array(y_test)
y_pred = np.array(y_pred)

cm = confusion_matrix(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

sn = cm[1][1]/(cm[1][1]+cm[1][0])
sp = cm[0][0]/(cm[0][0]+cm[0][1])

# plot(history)
print("\n %s, %s, %s, %s, %s \n" %(str(acc), str(mcc), str(sn), str(sp), cm))

101/101 [==============================] - 0s 4ms/step

 0.7885926844389337, 0.34839809143939426, 0.7747035573122529, 0.789774638412378, [[2348  625]
 [  57  196]] 



In [65]:
#stacked_model.save("selected_models/combined_feature_level_362_.h5")